# Package Config

In [44]:
from plotly.offline import init_notebook_mode, iplot # from plotly.offline import init_notebook_mode, iplot # 
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

import numpy as np
from scipy.integrate import odeint

# Load Models

In [45]:
%run AgentDynamics.ipynb

# Simulation

In [46]:
# System settings

n = 2
N = 3
nStates = n*2

AgentPars = {
    'n'  : n,
    'N'  : N,
    'k_i': 1,
    'm_i': 1,
}

agents = list(range(0,N))



# Time settings
dt    = 0.5                        # Networked Controller sampling time
odesteps_per_timesample = 10       # Number of ODE integration steps per discrete time sample
t_end = 20                         # Simulation time
t_discrete = np.arange(0,t_end,dt) # Discrete time sample vector

In [49]:
def get_s_dot(s,t,u):
    s_i = [s[i*nStates:(i+1)*nStates]  for i in agents]
    s_dot_i = [get_s_dot_i(t, s_i[i], u_i[i], AgentPars)   for i in agents]
    return np.concatenate(s_dot_i)

# Initial Conditions
s_start = 2.5 * np.random.randn(2*n*N) + 3

# Initialize empty result vectors
t_history = np.empty(shape=[1,      0 ]) 
s_history = np.empty(shape=[N*nStates, 0]) 
end = -1

# At every sample do
for tnow in t_discrete:
    
    # List of all the control inputs
    
     
    r_i = [s_start[2*i*n:(2*i+1)*n] for i in agents]
    
    u_i = [-1*r_i[i] for i in agents]
        
    t_sim = np.linspace(tnow,tnow+dt,odesteps_per_timesample)
    s_sim = odeint(get_s_dot, s_start, t_sim, args=(u_i,)).T
    
    ## The final y state is the initial state for next time
    s_start = s_sim[:,end]
       
    # Append the results
    t_history = np.append(t_history,t_sim)
    s_history = np.append(s_history,s_sim, axis = 1)

    
## Match time and output signals for plotting    
posx_plot = Scatter(x = t_history, y = s_history[0, :], name = 'position x_1')   
posy_plot = Scatter(x = t_history, y = s_history[1, :], name = 'position y_2')   



position_traces = [Scatter(x = s_history[i*nStates, :], y = s_history[i*nStates+1, :], name = 'Agent ' + str(i))  for i in agents]
    
# Plot results inline    
iplot([posx_plot,posy_plot])
iplot(position_traces)


print()